## load library

In [1]:
import pickle
import os 
import shutil
import pandas as pd
import re
import numpy as np

## load data

In [2]:
datadir = "/Users/saskia/unibe19/master_thesis/TKI_project/data/"
ds_dir = os.path.join(datadir, 'dataset_PC_HN')
info = pickle.load(open(os.path.join(ds_dir, "info.pckl"), "rb"))
info

{0: {'exp': 'LM',
  'stim': 'IL-1b High',
  'stim_class': 1,
  'stim_class_simple': 1,
  'bn': '1_il1b_1_s1c1.tif'},
 1: {'exp': 'LM',
  'stim': 'TNF',
  'stim_class': 3,
  'stim_class_simple': 2,
  'bn': '2_TNF_1_s1c1.tif'},
 2: {'exp': 'LM',
  'stim': 'IL-1b High',
  'stim_class': 1,
  'stim_class_simple': 1,
  'bn': '3_IL1B_1_s1.tif'},
 3: {'exp': 'LM',
  'stim': 'NS',
  'stim_class': 0,
  'stim_class_simple': 0,
  'bn': '4_NS_1_s1c1.tif'},
 4: {'exp': 'LM',
  'stim': 'NS',
  'stim_class': 0,
  'stim_class_simple': 0,
  'bn': '5_NS_1_s1c1.tif'},
 5: {'exp': 'LM',
  'stim': 'IL-1b High',
  'stim_class': 1,
  'stim_class_simple': 1,
  'bn': '6_IL1B_1_s1c1.tif'},
 6: {'exp': 'SS',
  'stim': 'IL-1b Low',
  'stim_class': 2,
  'stim_class_simple': 1,
  'bn': 'FC2_Il1b_low_1_s1c1.tif'},
 7: {'exp': 'SS',
  'stim': 'IL-1b High',
  'stim_class': 1,
  'stim_class_simple': 1,
  'bn': 'FC3_Il1b_high_1_s1c1.tif'},
 8: {'exp': 'SS',
  'stim': 'IL-1b Low',
  'stim_class': 2,
  'stim_class_simple':

get the *names* of folders that contain our two classes of interest:

In [3]:
folder_names = []
for i in info.keys():
    if info[i].get('stim') == 'IL-1b High' or info[i].get('stim') == 'NS' or info[i].get('stim') == 'TNF': 
        folder_names.append(i)

### get list of all folders and all image files 

In [4]:
folder_list = [os.path.join(ds_dir, f'{folder_name:03d}') for folder_name in folder_names] 
# 3 digits in total with 0 in front

In [5]:
img_names = []
folder_names = []
for folder in folder_list:
    for img in sorted(os.listdir(folder)): 
        img_name = os.path.join(folder, img)
        img_names.append(img_name)
        folder_names.append(folder)

In [6]:
len(img_names)

186

#### build dataframe 

In [7]:
dict_Df = pd.DataFrame.from_dict(info, orient='index') 
paths_Df = pd.DataFrame({'path_to_img': img_names, 'path_to_folder': folder_names}) 
# on part un peu du principe que les jolies listes sont dans le même ordre -> un peu risky... 

add image id (3 digits of folder + 3 digits of image) 

In [8]:
paths_Df['img_id'] = paths_Df.apply(lambda row: re.search(r'/(\d{3})/(\d{3})', row.path_to_img).group(1) + re.search(r'/(\d{3})/(\d{3})', row.path_to_img).group(2), axis=1)

- add group to ``paths_Df`` 
- add info of ``dict_Df`` to ``paths_Df`` by merging using group

In [9]:
paths_Df['group'] = paths_Df.apply(lambda row: int(re.search(r'/(\d{3})$', row.path_to_folder).group(1)), axis=1)

In [10]:
dict_Df['group'] = dict_Df.index

In [11]:
df = pd.merge(paths_Df, dict_Df, on='group')
df

,path_to_img,path_to_folder,img_id,group,exp,stim,stim_class,stim_class_simple,bn
0,/Users/saskia/unibe19/master_thesis/TKI_projec...,/Users/saskia/unibe19/master_thesis/TKI_projec...,000000,0,LM,IL-1b High,1,1,1_il1b_1_s1c1.tif
1,/Users/saskia/unibe19/master_thesis/TKI_projec...,/Users/saskia/unibe19/master_thesis/TKI_projec...,000001,0,LM,IL-1b High,1,1,1_il1b_1_s1c1.tif
2,/Users/saskia/unibe19/master_thesis/TKI_projec...,/Users/saskia/unibe19/master_thesis/TKI_projec...,000002,0,LM,IL-1b High,1,1,1_il1b_1_s1c1.tif
3,/Users/saskia/unibe19/master_thesis/TKI_projec...,/Users/saskia/unibe19/master_thesis/TKI_projec...,000003,0,LM,IL-1b High,1,1,1_il1b_1_s1c1.tif
4,/Users/saskia/unibe19/master_thesis/TKI_projec...,/Users/saskia/unibe19/master_thesis/TKI_projec...,000004,0,LM,IL-1b High,1,1,1_il1b_1_s1c1.tif
...,...,...,...,...,...,...,...,...,...
181,/Users/saskia/unibe19/master_thesis/TKI_projec...,/Users/saskia/unibe19/master_thesis/TKI_projec...,026001,26,SA,NS,0,0,210217_EXP3_NS_tdTomOTI_1_s1c1.tif
182,/Users/saskia/unibe19/master_thesis/TKI_projec...,/Users/saskia/unibe19/master_thesis/TKI_projec...,026002,26,SA,NS,0,0,210217_EXP3_NS_tdTomOTI_1_s1c1.tif
183,/Users/saskia/unibe19/master_thesis/TKI_projec...,/Users/saskia/unibe19/master_thesis/TKI_projec...,026003,26,SA,NS,0,0,210217_EXP3_NS_tdTomOTI_1_s1c1.tif
184,/Users/saskia/unibe19/master_thesis/TKI_projec...,/Users/saskia/unibe19/master_thesis/TKI_projec...,026004,26,SA,NS,0,0,210217_EXP3_NS_tdTomOTI_1_s1c1.tif


for each group:

- shuffle indices
- last 20% for test data 
- remaining 80% for train data 


create lists of indices for test and train

In [12]:
test_indices = []
train_indices = []

groups = df['group'].unique()

for group in groups: 
    df_temp = df[df['group'] == group]
    idxs = df_temp.index
    idxs = np.random.RandomState(seed=18).permutation(idxs) 
    n_test_data = int(len(idxs) * 0.2) 
    if n_test_data >= 1:
        test_indices.append(idxs[-n_test_data:])
        train_indices.append(idxs[:-n_test_data])
    else:
        train_indices.append(idxs)
        
test_indices = np.concatenate( test_indices, axis=0 )
train_indices = np.concatenate( train_indices, axis=0 )

create train and test dataframe with indices

In [13]:
test_df = df.loc[test_indices]
train_df = df.loc[train_indices]

### creating new directories

In [14]:
folder3classes = os.path.join(datadir, 'dataset3class_NS_IL1bHigh_TNF')
if not os.path.exists(folder3classes):
    os.mkdir(folder3classes)
train_folder = os.path.join(folder3classes, 'train')
if not os.path.exists(train_folder):
    os.mkdir(train_folder)
test_folder = os.path.join(folder3classes, 'test') 
if not os.path.exists(test_folder): 
    os.mkdir(test_folder)


create folder for classes:

In [15]:
classes3 = df['stim_class_simple'].unique()

for folder in [test_folder, train_folder]:

    for a_class in classes3:
        class_folder = os.path.join(folder, str(a_class))
        if not os.path.exists(class_folder): 
            os.mkdir(class_folder)

assign to group_folder - *copy images*

In [16]:
for df, folder in zip([train_df, test_df], [train_folder, test_folder]):
    for original_path, img_id, label in zip(df['path_to_img'], df['img_id'], df['stim_class_simple']): 
        dest_folder = os.path.join(folder, str(label), str(img_id) + '.png') # renamed with img_id 
        shutil.copy(original_path, dest_folder) 
